In [1]:
from datascience import *
import numpy as np
from math import *
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

## Lesson 32: Likelihood Ratio Tests

Last time, we introduced Likelihood Ratio tests. Recall that the point of a likelihood ratio test is to compare the likelihood function under a hypothesized value of the parameter with the liklihood function at its maximum. Instead of looking at the ratio $\Lambda$ itself, we often consider $-2\log \Lambda$ instead, since it has a handy distribution. 

### Example 1: Exponential Distribution

Suppose $X_1,X_2,...,X_n$ is an iid sequence of random variables from the exponential distribution with unknown parameter $\lambda$. Recall that the maximum likelihood estimate of $\lambda$ is $1\over\bar{X}$. We collect a random sample of size 20 and want to test the hypothesis $H_0: \lambda = 3$ vs $H_1: \lambda \neq 3$. Using the data in the python box below, conduct a likelihood ratio test on this hypothesis.  

In [2]:
my_data=np.array([0.18,0.277,0.105,0.126,0.225,0.026,0.123,0.423,0.006,0.281,0.050,0.692,0.105,0.275,0.346,0.079,0.045,0.222,0.063,0.281])

$f(x) = \lambda e^{-\lambda x}$

$l(\lambda|x) = \sum( log(f(x)) ) = \sum( log(\lambda e^{-\lambda x}) )$

$\sum( log(\lambda) + log(e^{-\lambda x_{i}}) ) $

$\sum( log(\lambda)) + \sum( -\lambda x_{i})  $

$n log(\lambda) -\lambda \sum( x_{i})  $

For $H_{0}$ : $l(\lambda|x) = 20 log(3) - 3 \sum( x_{i})$

For $H_{1}$ : $l(\lambda|x) = 20 log(\frac{1}{\bar{X}}) - \frac{1}{\bar{X}} \sum( x_{i})$

$l(\lambda|x) = 20 log(\frac{1}{\bar{X}}) - n = 20 log(\frac{1}{\bar{X}}) - 20$

In [3]:
xbar = np.mean(my_data)
xsum = sum(my_data)
test_stat = 2*(20*log(1/xbar) - 20 - 20*log(3)+3*xsum)
test_stat


4.719222360188461

In [4]:
#Now we want the probability of my data or more extreme using a chi distribution.
llambda = 1/xbar
print(llambda)
p = 1-stats.chi2.cdf(test_stat,1)
print('p = ',p)

5.089058524173028
p =  0.029827229194775096


Since p is less than 5%, we reject the null.

#### Power

Suppose that the true value of $\lambda$ is 5. Let's determine the power of this test. Let $n=20$. Then determine the power if $n=50$. Remember, power is the probability of correctly rejecting the null hypothesis. 

First, find what value of $-2 \log \Lambda$ would lead you to reject $H_0$. This is sometimes called the critical value. 

In [5]:
#This finds where 95% of the area is to the left, or 5% is to the right
#1 degree of freedom because we are only estimating 1 thing, lambda
crit = stats.chi2.ppf(0.95,1)
crit

3.841458820694124

Next, obtain the power. Obtain a sample of size 20 from the true population and obtain the value of $-2\log \Lambda$ for this sample. Repeat many times and determine how often you reject the null hypothesis. 

In [6]:
#Sample from an exponential with lambda = 5
T = []
EXP = stats.expon(scale = 1/5)
for n in np.arange(10000):
    A = EXP.rvs(size = 20)
    xbar = np.mean(A)
    xsum = sum(A)
    T = np.append(T,2*(20*log(1/xbar) - 20 - 20*log(3)+3*xsum))
    
power = sum(T>crit)/10000
power

0.5982

Repeat for a sample size of 50. What do you expect to happen to power? 

In [7]:
T = []
EXP = stats.expon(scale = 1/5)
for n in np.arange(10000):
    A = EXP.rvs(size = 50)
    xbar = np.mean(A)
    xsum = sum(A)
    T = np.append(T,2*(50*log(1/xbar) - 50 - 50*log(3)+3*xsum))
    
power = sum(T>crit)/10000
power

0.9508

We would expect exactly what happens, that our power increases with sample size. We are correct more often because we have more information.

### A Different Test

We've explored hypothesis tests in this class before. Taking advantage of our computing power, we don't have to rely on test statistics with asymptotic distributions. Let's conduct a more direct hypothesis test using simulation. Recall:

$$
H_0: \lambda = 3
$$

$$
H_1: \lambda \neq 3
$$

Pick a different test statistic. Obtain an empirical distribution of that test statistic under $H_0$. Next, find the $p$-value by determining how often this test statistic is at or further away from the test statistic derived from the sample. Remember that this is a two-sided test. 

In [33]:
#xbar
T = []
A = stats.expon(scale = 1/3)
for n in np.arange(10000):
    X = A.rvs(size= 20)
    T = np.append(T,np.mean(X))
    
p = sum(T>np.mean(my_data))/10000
1-p

0.019399999999999973

How did the $p$-value compare to the LRT $p$-value? I wonder how the power of this test compares to our LRT. 

We find that our sample data is on the low end, which could cause us to reject for a two-sided test. Only 1.94% of the time will a $\lambda$=3 yield a mean for a sample of size 20 that matches our data or more extreme. We therefore reject the null because 1.94<2.5. 

The LRT is more certain even though it has a value of 2%, because we reject at 5% for a one-sided test.

#### Power

Let's figure out the power of this test. First, determine for what values of the test statistic would lead us to reject $H_0$. These values can be referred to as your rejection region. 

In [46]:
Xbarbar = np.mean(T)
print(sum( (Xbarbar+.1425) > T )/10000)  
print(sum(T > (Xbarbar-.1425) )/10000) 
rejlow = Xbarbar-.1425
rejhigh = Xbarbar+.1425
print (rejlow)
print (rejhigh)
#Note that even though xbar should have a normal distribution, it is definitely not symmetric at a sample size of 20

0.9631
0.9853
0.19135427061025004
0.47635427061025004


Now, determine the power of this test. Like in the LRT case, obtain a sample of size 20 and obtain the test statistic. Repeat many times and see how often your test statistic is in your rejection region. 

In [47]:
T = []
EXP = stats.expon(scale = 1/5)
for n in np.arange(10000):
    A = EXP.rvs(size = 20)
    xbar = np.mean(A)
    T = np.append(T,xbar)
    
power = sum(T<rejlow)/10000 + sum(T>rejhigh)/10000
power

0.4637

Repeat for a sample size of 50. Note that you will have to obtain new critical values in order to do this.  

In [49]:
T = []
A = stats.expon(scale = 1/3)
for n in np.arange(10000):
    X = A.rvs(size= 50)
    T = np.append(T,np.mean(X))

In [61]:
Xbarr = np.mean(T)
sum(T < Xbarr -.1)/10000 + sum(T>Xbarr+.1)/10000
rejlow50 = Xbarr-.092
rejhigh50 = Xbarr+.092
#print (rejlow50)
#print (rejhigh50)
sum(T < Xbarr -.092)/10000 + sum(T>Xbarr+.092)/10000

0.050199999999999995

In [63]:
T = []
EXP = stats.expon(scale = 1/5)
for n in np.arange(10000):
    A = EXP.rvs(size = 50)
    xbar = np.mean(A)
    T = np.append(T,xbar)
    
power = sum(T<rejlow50)/10000 + sum(T>rejhigh50)/10000
power

0.9188

Note that the power nearly doubled with the increase in sample size, similar to the LRT. However, the power of the LRT was significantly higher in both cases.